
# 1. OpenAI VLM (GPT) - Basics
This section demonstrates the basic usage of OpenAI's Vision Language Model (VLM) capabilities using GPT-4.1.
We will use the OpenAI API to analyze an image and provide detailed textual insights.

**Support Material**

- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/guides/text
- https://platform.openai.com/docs/guides/images-vision?api-mode=chat
- https://platform.openai.com/docs/guides/structured-outputs


In [ ]:
import openai
from dotenv import load_dotenv  
import base64
import json
import textwrap
import os

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


load_dotenv()
# Instantiate client; API key read from environment (.env)
openAIclient = openai.OpenAI()


# Path to your image
img = "images/street_scene.jpg"




In [26]:
#basic call to gpt with prompt and image

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))


The image depicts a busy urban street scene with a mix of people and vehicles. In the foreground, there is a person
lying on the ground while another person sits cross-legged nearby using a smartphone or a similar device. Several
pigeons are scattered around them. On a bench close by, there is an older man in a suit and a younger woman reading a
newspaper. Another woman is walking by, holding a phone and a drink.   In the background, there is typical city traffic,
including a taxi, a car, a motorbike, and a person playing a guitar while walking. The buildings have classic and modern
architecture, and the overall atmosphere is lively with a mix of pedestrians and vehicles. The sun is either rising or
setting, casting a warm light over the scene.



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/structured-outputs


In [27]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={ "type": "json_object" },# NEW!!
    temperature = 0
)

returnValue = completion.choices[0].message.content


We parse the json in a dict structure:

In [28]:
output = json.loads(returnValue)
#json. loads() converts JSON strings to Python objects

So we can access specific infos:

In [29]:
output["foreground"]["people"][0]["description"]

'Young person with short hair, wearing a green jacket and shorts, focused on a tablet or phone'


# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide the json schema directly.



In [30]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",    
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
    temperature = 0
)

returnValue = completion.choices[0].message.content


In [31]:
output_image_extraction = json.loads(returnValue)


In [32]:
output_image_extraction["people"]

[{'position': 'sitting on the sidewalk',
  'age': 16,
  'activity': 'using a smartphone',
  'gender': 'male'},
 {'position': 'lying on the sidewalk',
  'age': 18,
  'activity': 'resting or sleeping',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 65,
  'activity': 'thinking or resting',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 25,
  'activity': 'reading a newspaper',
  'gender': 'female'},
 {'position': 'walking on the sidewalk',
  'age': 20,
  'activity': 'looking at a phone',
  'gender': 'female'},
 {'position': 'riding a motorcycle',
  'age': 30,
  'activity': 'driving',
  'gender': 'male'},
 {'position': 'walking on the street',
  'age': 28,
  'activity': 'playing guitar',
  'gender': 'male'},
 {'position': 'riding a scooter',
  'age': 27,
  'activity': 'driving',
  'gender': 'female'},
 {'position': 'walking on the sidewalk',
  'age': 35,
  'activity': 'walking',
  'gender': 'female'},
 {'position': 'inside a taxi car',
  'age': 40,
  'a

Alternatively: 


OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.


```python
from pydantic import BaseModel


class Person(BaseModel):
    position: str 
    age: int 
    activity: str 
    gender: str


class ImageExtraction(BaseModel):
    number_of_people: int 
    atmosphere: str 
    hour_of_the_day: int 
    people: list[Person] 

completion = openAIclient.beta.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": "describe the image in detail"}
    ],
    response_format=ImageExtraction,
)

output_image_extraction = completion.choices[0].message.parsed


We can then integrate the extracted information in full or partially in a new prompt for a new extraction

In [33]:
#alert service prompt 

alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_extraction)


In [34]:

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "user", "content": alert_prompt},
        {"role": "user", "content": alert_prompt}
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

No one appears to be in immediate danger based on the scene description. No alerts to either Child Hospital or normal
Hospital are necessary.


In [35]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Considering this list of people"+str(output_image_extraction["people"])+".Identify the youngest in the picture I provide and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width

response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

The youngest person in the list is a 16-year-old male sitting on the sidewalk using a smartphone. In the picture, he is
the person sitting on the ground near the bottom right corner, looking at a phone/tablet device.  Since you only
provided the image and list, I can't directly extract the exact coordinates. Please provide the bounding box or location
info, or a specific mark, and I will convert that into normalized coordinates for you. Alternatively, if you want me to
estimate his bounding box from the image, please confirm.



# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://ai.google.dev/gemini-api/docs/quickstart
- https://ai.google.dev/gemini-api/docs/text-generation
- https://ai.google.dev/gemini-api/docs/image-understanding
- https://ai.google.dev/gemini-api/docs/structured-output?example=recipe

In [ ]:
%matplotlib inline
from dotenv import load_dotenv  
from google import genai
from PIL import Image
import textwrap

import json


load_dotenv()
# Instantiate the Gemini client; the key is read from the environment (.env).
client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

# Path to your image
img = "images/street_scene.jpg"

Basic call:

In [37]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works to a 90 years old. in few words"
)

print(textwrap.fill(response.text, width=120))

Imagine a smart computer that *learns* from experience, much like a person does. It takes in lots of information, then
uses what it's learned to understand things, answer questions, or help you with tasks.


and with images: 

In [38]:
im = Image.open(img)

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details\n"],
                                          )

print(textwrap.fill(response.text, width=120))


This image captures a vibrant and somewhat surreal urban street scene during what appears to be either early morning or
late afternoon, given the warm, golden light bathing the cityscape. The scene is bustling with a mix of activity, from
fast-moving traffic to a variety of pedestrians engaged in different actions.  **Overall Composition and Lighting:** The
composition uses a wide-angle perspective, placing a prominent black and white striped crosswalk in the lower-center
foreground, acting as a visual anchor. The lighting is distinct, casting long shadows and illuminating the buildings and
street with a soft, warm glow, particularly noticeable on the brick buildings to the left and the distant skyline. This
golden hour light adds depth and atmosphere to the busy setting.  **Foreground - Sidewalk Activity:** On the immediate
left sidewalk, a large, rustic-looking wooden pot overflowing with bright red geraniums adds a splash of color. Nearby,
a young person with short brown hair, weari

Also here we can extract structured output (Gemini actually prefers pydantic syntax - let's see what happens with a schema as before)-> check limitations in https://ai.google.dev/gemini-api/docs/structured-output?example=recipe 

In [39]:
json_schema = {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]}}



config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details, follwoing exactly the given json schema\n"],
                                          config=config
                                          )



print(response.text)

{
  "scene_description": "A bustling city street scene bathed in warm, golden hour lighting, depicting various individuals engaged in different activities alongside moving vehicles and diverse architecture. The foreground features pedestrians on a crosswalk and sidewalk, while the background showcases a dynamic urban skyline.",
  "time_of_day": "Daytime, specifically late afternoon or early evening, indicated by the warm, golden glow of the sun.",
  "overall_mood": "Lively and dynamic, capturing a 'slice of life' in an urban setting with a mix of activity and repose.",
  "elements": [
    "people",
    "vehicles",
    "buildings",
    "natural elements",
    "street furniture and infrastructure"
  ],
  "details": [
    "A young person with short brown hair, wearing a green jacket and shorts, sits on the left side of the sidewalk, engrossed in a tablet or phone.",
    "A young man in a red hoodie and blue jeans lies flat on his back on the sidewalk near the center foreground, surrounded

Does it match your schema?

Let's try to use Gemini to detect an object in the image and get its coordinates:


In [40]:
prompt = "Identify the youngest in the picture and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."


config={"response_mime_type": "application/json"}

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[img, prompt],
                                          config=config
                                          )

bounding_boxes = json.loads(response.text)
print(bounding_boxes)


{'box_2d': [631, 287, 853, 396]}


Gemini2+ was trained specifically for object detection/ segmentation tasks. More details: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Spatial_understanding.ipynb


## 3.Extract Structured Infos from Hand-written note - GPT & Gemini

Let’s try **not** to extract structured information from a handwritten note (e.g., `prescription1.jpg`) using **both models**.

Consider the file: `/images/prescription1.jpg`.  
Have a look at it.

### JSON Schema
Let’s define a JSON schema for the extraction task:


In [41]:
json_schema_prescription = {
 "name": "prescription_extract",
"schema": {
  "type": "object",
  "properties": {
    "doctor_name": { "type": "string" },
    "patient_name": { "type": "string" },
    "patient_dob": { "type": "string" },
    "meds": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "dose": { "type": "string" },
          "frequency": { "type": "string" },
          "instructions": { "type": "string" }
        },
        "required": ["name"]
      }
    },
    "signature": { "type": "boolean" }
  },
  "required": ["doctor_name", "patient_name", "meds"]
}}

Extract structured infos using Gemini: 

In [42]:
im = Image.open("images/prescription1.jpg")

config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema_prescription,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Extract infos from image, follwoing the given json schema.\n"],
                                          config=config
                                          )



print(response.text)

{
  "doctor_name": "Dr. Markus Hütter",
  "doctor_specialty": "Chirurg",
  "patient_name": "Claudie Fischer",
  "patient_dob": "1.4.1978",
  "patient_gender": "f",
  "medication": "Ibuprofen",
  "dosage": "3x 400mg",
  "instructions": "nach dem Essen"
}


If the output is **not valid JSON** and contains extra strings, it must be **parsed** before it can be loaded into a Python dict.  
Below is an example helper function that does this.

> **Note:** Since Gemini returns a Pydantic model, you *could* use Pydantic methods to handle parsing.  
> We avoid that here to keep the workflow generally compatible across models.


In [43]:
import re
import json 
def parse_json_in_output(output):
    """
    Extracts and converts JSON-like data from the given text output to a Python dictionary.
    
    Args:
        output (str): The text output containing the JSON data.
    
    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    # Regex to extract JSON-like portion
    json_match = re.search(r"\{.*?\}", output, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        # Fix single quotes and ensure proper JSON formatting
        json_str = json_str.replace("'", '"')  # Replace single quotes with double quotes
        try:
            # Convert the fixed JSON string into a dictionary
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError:
            return "The extracted JSON is still not valid after formatting."
    else:
        return "No JSON data found in the given output."

In [44]:
#print(parse_json_in_output(response.text))


In [45]:
json.loads(response.text)

{'doctor_name': 'Dr. Markus Hütter',
 'doctor_specialty': 'Chirurg',
 'patient_name': 'Claudie Fischer',
 'patient_dob': '1.4.1978',
 'patient_gender': 'f',
 'medication': 'Ibuprofen',
 'dosage': '3x 400mg',
 'instructions': 'nach dem Essen'}

Now let's do the same with GPT

In [46]:
im = "images/prescription1.jpg"

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(im)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",   "json_schema": json_schema_prescription},
    temperature = 0
)

returnValue = completion.choices[0].message.content

In [47]:
returnValue

'{"doctor_name":"Dr. Markus Müller","patient_name":"Claudia Fischer","patient_dob":"1.4.1978","meds":[{"name":"Ibuprofen","dose":"400 mg","frequency":"3x","instructions":"nach dem Essen"}],"signature":true}'

Any difference wiht the output of Gemini vs your schema? 

No need for parsing now. We load the json in a python dict structure with json.loads

In [48]:
print(json.loads(returnValue))

{'doctor_name': 'Dr. Markus Müller', 'patient_name': 'Claudia Fischer', 'patient_dob': '1.4.1978', 'meds': [{'name': 'Ibuprofen', 'dose': '400 mg', 'frequency': '3x', 'instructions': 'nach dem Essen'}], 'signature': True}
